In [53]:
from openai import OpenAI
from dotenv import load_dotenv
import os

In [54]:
load_dotenv()

True

In [55]:
apikey = os.getenv("OPENAI_API_KEY")
client = OpenAI(
    api_key=apikey
)
print(client)

### `stream_chatbot(message)`

This function interacts with the chatbot using stream functionality.

**Parameters:**
- `message`: The message sent by the user to the chatbot.

**Returns:**
- None

**Functionality:**
- Sends the user message to the chatbot.
- Receives and prints the response from the chatbot.

In [56]:
def stream_chatbot(message):
    chat_completion_stream = client.chat.completions.create(
        messages=[
            {"role":"user",
             "content":message,
             },
        ],
        model="gpt-3.5-turbo",
        stream=True
    )
    for chunk in chat_completion_stream:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="")

### `prompt_user()`

This function prompts the user for input.

**Parameters:**
- None

**Returns:**
- `user_input`: The input provided by the user.

**Functionality:**
- Displays a prompt for the user to input a message.
- Returns the user's input.

In [57]:
def prompt_user():
    user_input = input("You: ")
    return user_input

### `print_user_message(user_message)`

This function prints the user's message.

**Parameters:**
- `user_message`: The message provided by the user.

**Returns:**
- None

**Functionality:**
- Prints the user's message in the format: "You: [user_message]".


In [58]:
def print_user_message(user_message):
    print("You: ", user_message)
    

### `chatbot(user_message)`

This function interacts with the chatbot based on user input.

**Parameters:**
- `user_message`: The message provided by the user.

**Returns:**
- None

**Functionality:**
- Initiates interaction with the chatbot by passing the user's message.
- Prints the chatbot's response.


In [59]:
def chatbot(user_message):
    print("Chatbot: ")
    stream_chatbot(user_message)

In [52]:
prompt = prompt_user()
print_user_message(prompt)
chatbot(prompt)

You:  Hiiii, it's me. Akuske!
Chatbot: 
Hello Akuske! How can I assist you today?